In [1]:
from ECHO_modules.make_data_sets import make_data_sets # Import relevant module

In [19]:
import numpy as np
import pandas as pd

In [ ]:
year_range = [y for y in range(2004, 2022)]
data_sets = make_data_sets(['Toxic Releases', 'RCRA Violations']) # Create data sets for the specified year range
erie_toxic_releases = data_sets['Toxic Releases'].store_results(
    region_type="County", region_value=["ERIE"], state="NY", years=[2004, 2021],)
 # Show the results as a dataframe

erie_rcra_violations = data_sets["RCRA Violations"].store_results(
    region_type="County", region_value=["ERIE"], state="NY", years=[2004, 2021],)
# erie_rcra_violations.dataframe['VIOLATION_TYPE_DESC'].unique() # Show

Data last modified: 2025-03-30 00:00:00
There were 1553 program records found
Data last modified: 2025-03-30 00:00:00
There were 682 program records found


array(['Standards Applicable to Generators of HW: Pre-Transport Requirements Applicable to Small and Large Quantity Generators',
       'Standards for Universal Waste Management: Standards for Small Quantity Handlers ',
       'Standards Applicable to Generators of HW: General',
       'ID and Listing of HW: General',
       'Standards Applicable to Generators of HW: Manifest Requirements Applicable to Small and Large Quantity Generators',
       'State Statutory or Regulatory requirements that are broader-in-scope than the federal RCRA requirements',
       'Standards for Used Oil: Generators',
       'Interim Status Standards for Owners and Operators of HW TSDs: Preparedness and Prevention',
       'Interim Status Standards for Owners and Operators of HW TSDs: General Facility Standards',
       'Interim Status Standards for Owners and Operators of HW TSDs: Contingency Plan and Emergency Procedures',
       'Interim Status Standards for Owners and Operators of HW TSDs: Use and Manage

In [20]:
np.sort(erie_toxic_releases.dataframe.POLLUTANT_NAME.unique())

array(['1,1,2-Trichloroethane', '1,2,4-Trimethylbenzene', '1-Butanol',
       '2,4-Dimethylphenol', '3-Chloro-2-methyl-1-propene',
       'Acetaldehyde', 'Alumina', 'Ammonia', 'Anthracene', 'Antimony',
       'Barium compounds -- TRI ', 'Benzene', 'Benzo[ghi]perylene',
       'Bifenthrin', 'Butyl acrylate', 'Carbon disulfide',
       'Certain glycol ethers, a limited set -- TRI', 'Chlorine',
       'Chromium', 'Chromium compounds -- TRI', 'Cobalt',
       'Cobalt compounds', 'Copper', 'Copper compounds -- TRI',
       'Creosote', 'Cresol', 'Cumene', 'Cyclohexane',
       'Di(2-ethylhexyl) phthalate', 'Dibenzofuran',
       'Diisocyanates -- TRI', 'Dioxin and dioxin-like compounds -- TRI',
       'Ethylbenzene', 'Ethylene', 'Ethylene glycol', 'Formaldehyde',
       'Formic acid', 'Hexane', 'Hydrochloric acid', 'Hydrofluoric acid',
       'Hydrogen sulfide', 'Lead', 'Lead compounds', 'Manganese',
       'Manganese compounds', 'Mercury', 'Mercury compounds', 'Methanol',
       'Methyl iso